In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
def change_label(df):
  df.label.replace(['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm'],'Dos',inplace=True)
  df.label.replace(['ftp_write','guess_passwd','httptunnel','imap','multihop','named','phf','sendmail',
       'snmpgetattack','snmpguess','spy','warezclient','warezmaster','xlock','xsnoop'],'R2L',inplace=True)
  df.label.replace(['ipsweep','mscan','nmap','portsweep','saint','satan'],'Probe',inplace=True)
  df.label.replace(['buffer_overflow','loadmodule','perl','ps','rootkit','sqlattack','xterm'],'U2R',inplace=True)

def normalization(df,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
  return df

In [ ]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty_level"]

In [ ]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Tdjaaleb/IDS_AdversarialML/main/Data/KDDTrain%2B.txt',header=None, names=col_names)
df_test = pd.read_csv('https://raw.githubusercontent.com/Tdjaaleb/IDS_AdversarialML/main/Data/KDDTest%2B.txt',header=None, names=col_names)

In [ ]:
df = pd.concat([df_train, df_test], ignore_index=True)
print("Train : ", df_train.shape)
print("Test : ", df_test.shape)
print("Global : ", df.shape)

In [ ]:
df.drop(['difficulty_level'],axis=1,inplace=True)
change_label(df)
numeric_col = df.select_dtypes(include='number').columns
std_scaler = StandardScaler()
df = normalization(df.copy(),numeric_col)
cat_col = ['protocol_type','service','flag']
categorical = df[cat_col]
categorical = pd.get_dummies(categorical,columns=cat_col)

multi_df = df.copy()
multi_label = pd.DataFrame(multi_df.label)
le2 = preprocessing.LabelEncoder()
enc_label = multi_label.apply(le2.fit_transform)
multi_df['intrusion'] = enc_label
multi_df = pd.get_dummies(multi_df,columns=['label'],prefix="",prefix_sep="") 
multi_df['label'] = multi_label

numeric_multi = multi_df[numeric_col]
numeric_multi['intrusion'] = multi_df['intrusion']
corr = numeric_multi.corr()
corr_y = abs(corr['intrusion'])
highest_corr = corr_y[corr_y >0.5]
highest_corr.sort_values(ascending=True)
numeric_multi = multi_df[['count','logged_in','srv_serror_rate','serror_rate','dst_host_serror_rate',
                        'dst_host_same_srv_rate','dst_host_srv_serror_rate','dst_host_srv_count','same_srv_rate']]
numeric_multi = numeric_multi.join(categorical)
multi_df = numeric_multi.join(multi_df[['intrusion','Dos','Probe','R2L','U2R','normal','label']])

In [ ]:
df_train = multi_df[:125973]
df_test = multi_df[125973:]

In [ ]:
print("Train : ", df_train.shape)
print("Test : ", df_test.shape)

In [ ]:
number_of_chunks = 2

for id, df_i in  enumerate(np.array_split(df_train, number_of_chunks)):
    df_i.to_csv(f'Data/df_train_{id}.csv')

df_test.to_csv("Data/df_test.csv")